# Transforming Images
In the previous section, we discussed the principles of affine and rigid-body transformations and saw how converting between voxel-space and world-space can be conceptualised as an affine transformation of the image coordinates. However, the utility of these forms of transforms is not just moving between different spaces, but also in providing a framework for transforming the location of the brain within an image. Generally speaking, because we have a choice of two coordinate spaces, we also have a choice of performing a transformation in *world-space* or in *voxel-space*. As we will see, world-space transformations alter the rules for mapping voxels to millimetres, without changing the voxel values. In comparison, voxel-space transformations directly alter the voxels in the image. In this section, we will explore the principles and utility of each type of transformation.

## Transforming an Image in World-space
One of the main motivating factors behind a world-space representation of an image is so that we can create more intuitive transformations for moving the brain. Generally speaking, transformations in world-space make more sense because we can use real-world units. For instance, we can move the brain back by 3mm and up by 5mm. To do this, we have to combine the $\mathbf{M}$ matrix from the header with another affine transform matrix $\mathbf{T}$. If we use the following as our example $\mathbf{M}$ matrix

$$
\mathbf{M} = 
\begin{bmatrix}
-3 & 0 & 0 &  81  \\ 
 0 & 3 & 0 & -115 \\ 
 0 & 0 & 3 & -73  \\
 0 & 0 & 0 &  1
\end{bmatrix}
$$

we can see the original millimetre location of the voxel coordinates $\mathbf{a} = [20, 20, 20, 1]'$ is given by

$$
\mathbf{Ma} = 
\begin{bmatrix} 
 21 \\
-55 \\ 
-13 \\
1 
\end{bmatrix}
$$

Focusing on the $y$-axis, the voxel index of 20 is mapped to -55mm from the origin in world-space. If we wanted to move the image 9mm *backwards*, we could combine $\mathbf{M}$ with a transformation matrix $\mathbf{T}$ that encodes a negative 9mm shift along the $y$-axis

$$
\mathbf{T} = 
\begin{bmatrix}
 1 & 0 & 0 &  0 \\ 
 0 & 1 & 0 & -9 \\ 
 0 & 0 & 1 &  0 \\
 0 & 0 & 0 &  1
\end{bmatrix}
$$

Pre-multiplying $\mathbf{M}$ by $\mathbf{T}$ creates a new matrix that changes how the voxel coordinates are mapped into world-space. So, if we do the following

$$
\left(\mathbf{TM}\right)\mathbf{a} = 
\begin{bmatrix} 
 21 \\
-64 \\ 
-13 \\
1 
\end{bmatrix}
$$

we can see that the voxel coordinate of $y = 20$ is now mapped to -64mm from the origin, shifting it 9mm further *backwards*. If we want to make this shift permanent, we can create a new matrix $\mathbf{M}^{\ast} = \mathbf{TM}$ and then replace the original $\mathbf{M}$ matrix in the header with $\mathbf{M}^{\ast}$. The image is then forever mapped 9mm further backwards in world-space.

An important point about this process is that it has *not* altered the voxel data. All we did is change how the voxels coordinates are mapped to millimetre coordinates. In effect, we have changed the rules, but have not done anything to the actual image. Because of this, the image will look no different in voxel-space, but will be moved back 9mm in world-space. This change is somewhat invisible because it will not look like the image has been altered. You would only notice a difference if you saw the image displayed in world-space *and* had some reference point to compare it with. Typically this reference point is another image, as the point of world-space transforms is often to bring multiple images into alignment without changing the voxel data. This will make more sense next week when we discuss image registration. For now, just try to grasp the idea of world-space transformations, knowing that this will be further contextualised later.

## Transforming an Image in Voxel-space

As highlighted above, transforming and replacing the header matrix will not alter the voxel data. If we wanted this transformation to change the actual image data, we would need to turn this into a *voxel-space* transformation. The motivation for this is usually in the context of image registration and so will become clearer next week. The short version is that we usually map between different images in world-space, but once we have transformed the actual image data we do not need to perform this mapping anymore and can work exclusively in voxel-space. Computationally, this is much more efficient because we do not need to keep converting coordinates between different images. You can therefore think of the voxel-space transforming as *applying* the world-space transformation to the actual data.

Converting a world-space transformation into a voxel-space transformation can be achieved by defining the following mapping matrix

$$
\mathbf{Q} = (\mathbf{TM})^{-1}\mathbf{M}.
$$

$\mathbf{Q}$ can then be used to change any voxel coordinates in the original image into new voxel coordinates in the transformed image. For instance, using the $\mathbf{a}$ defined earlier

$$
\mathbf{Qa} = 
\begin{bmatrix}
20 \\
23 \\
20 \\
1
\end{bmatrix}.
$$

So, the original voxel coordinate of $y = 20$ has been transformed into a new voxel coordinate of $y = 23$. This works by taking the original voxel coordinates, transforming them into millimetres, applying the transformation in world-space and then transforming the shifted world-space coordinates back into voxels.

In order to use $\mathbf{Q}$ to actually change the voxel data, we would first create a blank image with the same dimensions as the original image. We would then work through each voxel in the new image and transform the corresponding voxel coordinates using $\mathbf{Q}$. These transformed coordinates tell us the coordinates of the voxel we should copy from the original image. In the example above, when we get to voxel $[20, 20, 20]$ in the new image we would use $\mathbf{Q}$ to transform these coordinates into $[20, 23, 20]$. This would tell us that we need to copy the value of voxel $[20, 23, 20]$ from the old image into voxel $[20, 20, 20]$ in the new image. This is conceptualised as a *pulling* operation, where the original data are pulled into their new locations.

This process is illustrated in the video below for a rotation operation. For each position in the new image on the left, we take the coordinates and transform them using $\mathbf{Q}$. This gives the coordinates illustrated in the image on the right. The voxel value associated with this coordinate is copied from the old image into the new image. If the transformation results in coordinates that are out-of-bounds, a value of -1 is inserted into the new image instead.

<div style="max-width: 1280px"><div style="position: relative; padding-bottom: 56.25%; height: 0; overflow: hidden;"><iframe src="https://livemanchesterac.sharepoint.com/sites/UOM-FBMH-MSc-NCCN/_layouts/15/embed.aspx?UniqueId=f519fafa-9b1e-4d90-8cca-9c2b5d731ce2&embed=%7B%22hvm%22%3Atrue%2C%22ust%22%3Atrue%7D&referrer=StreamWebApp&referrerScenario=EmbedDialog.Create" width="1280" height="720" frameborder="0" scrolling="no" allowfullscreen title="Rotate-Image.mp4" style="border:none; position: absolute; top: 0; left: 0; right: 0; bottom: 0; height: 100%; max-width: 100%;"></iframe></div></div>

## Transforming Images in SPM